In [1]:
%matplotlib inline
import requests
import pandas as pd
import numpy as np
from pandas import json_normalize 
from urllib.parse import urlencode
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import re
import json
#nltk.download('stopwords')

# 1

In [2]:
API_BASE_URL = "https://api.hh.ru/vacancies/"
DICT_KEYS = (
        "Ids",
        "Employer",
        "area",
        "address",
        "Name",
        "currency",
        "From",
        "To",
        "Experience",
        "Schedule",
        "Keys",
        "Description",
    )
rates = {
    "USD": 0.016,
    "EUR": 0.016,
    "UAH": 0.6,
    "RUR": 1
  }
query={"text": "python разработчик", "per_page": 10}

In [3]:

def сlean_tags(html_text: str) -> str:
    pattern = re.compile("<.*?>")
    return re.sub(pattern, "", html_text)    
    
    

In [4]:
target_url = API_BASE_URL + "?" + urlencode(query)
num_pages = requests.get(target_url).json()["pages"]
num_vacancies = requests.get(target_url).json()["found"]
print(str(num_pages)+'\n'+str(num_vacancies))

200
4760


### Collect vacancy IDs

In [5]:
ids = []
for idx in range(num_pages + 1):
    response = requests.get(target_url, {"page": idx})
    data = response.json()
    if "items" not in data:
        break
    ids.extend(x["id"] for x in data["items"])

In [6]:
def get_vacancy(vacancy_id: str):
    url = f"{API_BASE_URL}{vacancy_id}"
    vacancy = requests.api.get(url).json()
    salary = vacancy.get("salary")
    slr = {"currency":None,"from": None, "to": None}
    if salary:
         for k, v in slr.items():
            if vacancy["salary"][k] is not None:
                slr[k] = salary[k]
    return (            
        vacancy_id,
        vacancy["employer"]["name"],
        vacancy["area"]["name"],
        vacancy["address"],
        vacancy["name"],
        slr["currency"],
        slr["from"],
        slr["to"],
        vacancy["experience"]["name"],
        vacancy["schedule"]["name"],
        [el["name"] for el in vacancy["key_skills"]],
        сlean_tags(vacancy["description"])
    )

In [7]:
jobs_list = []
with ThreadPoolExecutor(max_workers=1) as executor:
    for vacancy in tqdm(
        executor.map(get_vacancy, ids),
        desc="Get data via HH API",
        ncols=12,
        total=len(ids),
    ):
        #print(vacancy)
        jobs_list.append(vacancy)

unzipped_list = list(zip(*jobs_list))

Get data via


In [8]:
result = {}
for idx, key in enumerate(DICT_KEYS):
    result[key] = unzipped_list[idx]

In [9]:
df = pd.DataFrame(data = result)

In [10]:
df.head()

,Ids,Employer,area,address,Name,currency,From,To,Experience,Schedule,Keys,Description
0,73112663,"ЛОТОС, инжиниринговая компания",Екатеринбург,None,Backend-разработчик на Python,RUR,60000.0,NaN,От 1 года до 3 лет,Удаленная работа,"[Python, HTML, CSS, Bootstrap, MySQL, Git, Fla...",— Вы хотите принять участие в реализации проек...
1,70705466,Фабрика Решений,Москва,None,"Python-разработчик (Django, DRF)",RUR,60000.0,220000.0,От 3 до 6 лет,Удаленная работа,"[Python, Django Framework, PostgreSQL, SQL, CS...","Привет! Мы, студия fabrique, ищем классного ju..."
2,73095539,Богач Евгений Александрович,Москва,"{'city': 'Люберцы', 'street': 'микрорайон Горо...",Программист Python,RUR,60000.0,100000.0,От 1 года до 3 лет,Полный день,"[Python, SQL, Linux, PostgreSQL, Git, ООП, NoSQL]","Требуется:программист в отдел разработки, плат..."
3,72313102,Рекруто,Санкт-Петербург,"{'city': 'Санкт-Петербург', 'street': 'набереж...",Intern / Junior Python backend developer,RUR,NaN,60000.0,Нет опыта,Гибкий график,"[Python, Celery, Docker, JavaScript, RabbitMQ,...","Python был, есть и будет, и у вас появилась ре..."
4,71681150,Бизнес-Азимут,Москва,None,Python-разработчик (Middle+/Senior),RUR,320000.0,350000.0,От 3 до 6 лет,Удаленная работа,"[Python, Hadoop, Spark, Hive, SQL]",Наша компания специализируется на аутсорсинге ...


In [11]:
display(df.shape)
display(df.info())

(2000, 12)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Ids          2000 non-null   object 
 1   Employer     2000 non-null   object 
 2   area         2000 non-null   object 
 3   address      839 non-null    object 
 4   Name         2000 non-null   object 
 5   currency     653 non-null    object 
 6   From         542 non-null    float64
 7   To           437 non-null    float64
 8   Experience   2000 non-null   object 
 9   Schedule     2000 non-null   object 
 10  Keys         2000 non-null   object 
 11  Description  2000 non-null   object 
dtypes: float64(2), object(10)
memory usage: 187.6+ KB


None

In [12]:
df.isna().mean().sort_values(ascending = False)

To             0.7815
From           0.7290
currency       0.6735
address        0.5805
Ids            0.0000
Employer       0.0000
area           0.0000
Name           0.0000
Experience     0.0000
Schedule       0.0000
Keys           0.0000
Description    0.0000
dtype: float64

In [13]:
print('Количество вакансий: {}'.format(df['Ids'].count()))

Количество вакансий: 2000


## Формируем json

In [14]:
df['salary'] = 'От '+ df['From'].astype(str)+ ' до '+ df['To'].astype(str) +' '+ df['currency']

In [15]:
df['salary'].value_counts()

От 200000.0 до nan RUR         25
От 150000.0 до nan RUR         22
От 100000.0 до 180000.0 RUR    17
От 100000.0 до nan RUR         16
От 2500.0 до 3500.0 USD        14
                               ..
От 25000.0 до nan RUR           1
От 200000.0 до 300000.0 RUR     1
От 55000.0 до 200000.0 RUR      1
От 130000.0 до 200000.0 RUR     1
От 1200.0 до 1600.0 USD         1
Name: salary, Length: 274, dtype: int64

In [16]:
df.rename(columns={"Name":"title","Experience":"work experience","area":"region"},inplace = True)

In [17]:
cols = [
    'title',
    'work experience',
    'salary',
    'region',
]

In [18]:
df[cols]

,title,work experience,salary,region
0,Backend-разработчик на Python,От 1 года до 3 лет,От 60000.0 до nan RUR,Екатеринбург
1,"Python-разработчик (Django, DRF)",От 3 до 6 лет,От 60000.0 до 220000.0 RUR,Москва
2,Программист Python,От 1 года до 3 лет,От 60000.0 до 100000.0 RUR,Москва
3,Intern / Junior Python backend developer,Нет опыта,От nan до 60000.0 RUR,Санкт-Петербург
4,Python-разработчик (Middle+/Senior),От 3 до 6 лет,От 320000.0 до 350000.0 RUR,Москва
...,...,...,...,...
1995,Программист,От 1 года до 3 лет,NaN,Екатеринбург
1996,Программист (Bitrix),От 1 года до 3 лет,NaN,Москва
1997,Ведущий разработчик чат-бота,От 3 до 6 лет,NaN,Москва
1998,Фронтенд разработчик,От 1 года до 3 лет,От 100000.0 до nan RUR,Краснодар


In [19]:
result = df[cols].to_json(orient="table",force_ascii=False,path_or_buf=r'hh_python.json',index=False)

# 2

In [20]:
def palindrom(s: str):
    s = s.replace(' ','')
    print(s)
    l = len(s)
    s_out = s[::-1] 
    print(s_out)
    if s_out == s:
        return True
    else:
        return False
        

In [21]:
check = ['taco cat','rotator','black cat']
for c in check:
    print(palindrom(c))

tacocat
tacocat
True
rotator
rotator
True
blackcat
tackcalb
False


# 3

In [22]:
def converter(n):
    result = ''
    for arabic, roman in zip((1000, 900, 500, 400, 100, 90, 50, 40, 10, 9, 5, 4, 1),
                             'M     CM   D    CD   C    XC  L   XL  X   IX V  IV I'.split()):
        result += n // arabic * roman
        n %= arabic
        #print('({}) {} => {}'.format(roman, n, result))
    return result

In [23]:
check = [3,9,1945]
for c in check:
    print(converter(c))

III
IX
MCMXLV


# 4

In [34]:
def checkio(in_str: str):
    open_s = ['[','(','{']
    close_s = [']',')','}']
    i_cls = 0
    i_opn = 0
    stack= []
    for i in in_str:
        if i in open_s:
            stack.append(i)
            i_opn +=1
        elif i in close_s:
            pos = close_s.index(i)
            if ((len(stack) > 0) and (open_s[pos] == stack[len(stack)-1])):
                stack.pop()
                i_cls +=1
            else:
                return False
    if i_cls != i_opn:
        return False   
    if len(stack) == 0:
        return True
    
print (checkio("[{}({})]")) 
print (checkio("{]")) 
print (checkio("{{}"))
print (checkio("{"))
print (checkio("}{"))
print (checkio("{}{"))
print (checkio("((){})"))
print (checkio("((){)}"))

True
False
False
False
False
False
True
False


# 5

In [57]:
def multiplier(x1: str, x2: str):
    a1 = int(x1,2)
    a2 = int(x1,2)
    return str(bin(a1*a2))

In [58]:
multiplier('111','101')

'0b110001'